In [4]:
!pip install pypdf langchain-openai langchain-core langchain-community langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 15.9 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os

os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_EMBEDDING_MODEL'] = userdata.get('OPENAI_EMBEDDING_MODEL')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['DART_API_KEY'] = userdata.get('DART_API_KEY')

In [25]:
# 필요한 라이브러리
import pandas as pd
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
import os

# 환경변수에서 API 키
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# 설정
PINECONE_INDEX_NAME = "company-rag"
PINECONE_DIMENSION = 1536
PINECONE_REGION = "us-east-1"
PINECONE_CLOUD = "aws"
EMBEDDING_MODEL = "text-embedding-3-small"

In [6]:
from google.colab import files
uploaded = files.upload()

Saving 통합_사업보고서 (2022-2025) 20 기업.csv to 통합_사업보고서 (2022-2025) 20 기업.csv


In [9]:
df = pd.read_csv('/content/통합_사업보고서 (2022-2025) 20 기업.csv')

print(df.columns)

Index(['년도', '회사명', '텍스트 미리보기'], dtype='object')


In [67]:
import pandas as pd
import re

df = pd.read_csv('/content/통합_사업보고서 (2022-2025) 20 기업.csv')
text_col = '텍스트 미리보기'

def remove_special(text):
    if pd.isnull(text):
        return ''
    text = str(text)
    # 1. 이스케이프 문자(\\n, \\t) → 실제로 변환 (줄바꿈은 유지)
    text = text.replace('\\n', '\n').replace('\\t', '\t')
    # 2. 특수문자 제거: 한글, 영문, 숫자, 공백, 마침표, 콤마만 남김
    text = re.sub(r'[^ㄱ-ㅎ가-힣a-zA-Z0-9.,\s]', ' ', text)
    # 3. 반복 공백 하나로
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['정제본문'] = df[text_col].apply(remove_special)

print(df[['회사명', '년도', text_col, '정제본문']].head(3))

df.to_csv('company_report.csv', index=False)


    회사명    년도                                           텍스트 미리보기  \
0  삼성전자  2022  사업보고서\n5.0\n삼성전자주식회사\nC\nA\n130111-0006246\n사 ...   
1  삼성전자  2023  사업보고서\n5.5\n삼성전자주식회사\nC\nA\nY\n130111-0006246\...   
2  삼성전자  2024  연결감사보고서\n6.0\n삼성전자(주)\n00260116\n100000000000\...   

                                                정제본문  
0  사업보고서 5.0 삼성전자주식회사 C A 130111 0006246 사 업 보 고 ...  
1  사업보고서 5.5 삼성전자주식회사 C A Y 130111 0006246 사 업 보 ...  
2  연결감사보고서 6.0 삼성전자 주 00260116 100000000000 51453...  


In [68]:
# 예시: 임의로 30개 줄 미리보기
for text in df['정제본문'].sample(30):
    print(text[:300])
    print('-' * 40)


사업보고서 5.0 삼성물산 주 C A 110111 0015762 사 업 보 고 서 제 59 기 사업연도 2022년 01월 01일 부터 2022년 12월 31일 까지 금융위원회 한국거래소 귀중 2023년 3월 9일 제출대상법인 유형 주권상장법인 면제사유발생 해당사항 없음 회 사 명 삼성물산주식회사 대 표 이 사 고 정 석, 오 세 철, 한 승 환 본 점 소 재 지 서울특별시 강동구 상일로 6길 26 상일동 전 화 02 2145 5114 홈페이지 http www.samsungcnt.com 작 성 책 임 자 직 책 경영기획실장 성 명 송
----------------------------------------
사업보고서 5.1 에이치엠엠 주식회사 C A 110111 0193740 정 정 신 고 보고 2023년 7월 14일 1. 정정대상 공시서류 2022년 사업보고서 2. 정정대상 공시서류의 최초제출일 2023년 3월 23일 3. 정정사항 항 목 정정요구 명령관련 여부 정정사유 정 정 전 정 정 후 V. 회계감사인의 감사의견 등1. 외부감사에 관한 사항나. 별도재무제표에 대한 회계감사인의 명칭 및 감사의견 2022년 사업보고서 재무사항 점검결과 반영 주1 주2 나. 별도재무제표에 대한 회계감사인의 명칭 및 감사의견 사업연도 감사인 감사의견
----------------------------------------
사업보고서 5.5 현대자동차 C A Y 사 업 보 고 서 제 56 기 사업연도 2023년 01월 01일 부터 2023년 12월 31일 까지 금융위원회 한국거래소 귀중 2024년 3월 13일 제출대상법인 유형 주권상장법인 면제사유발생 해당사항 없음 회 사 명 현대자동차 주식회사 대 표 이 사 장재훈 본 점 소 재 지 서울시 서초구 헌릉로 12 양재동 전 화 02 3464 1114 홈페이지 http www.hyundai.com 작 성 책 임 자 직 책 기획재경본부장 성 명 이승조 전 화 02 3464 2246 목 차 대표이사 등의 확인
---------------

In [69]:
import time
from openai import RateLimitError


CSV_PATH = "/content/company_report.csv"

# 1. Pinecone 초기화 및 인덱스 생성
pc = Pinecone(api_key=PINECONE_API_KEY)
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=PINECONE_DIMENSION,
        metric="cosine",
        spec=ServerlessSpec(cloud=PINECONE_CLOUD, region=PINECONE_REGION)
    )

# 2. CSV 로드 및 텍스트 변환
df = pd.read_csv(CSV_PATH)
texts = []

for idx, row in df.iterrows():
    row_text = "\n".join([f"{col}: {row[col]}" for col in df.columns if pd.notna(row[col])])
    texts.append(Document(page_content=row_text))

# 3. 텍스트 청크 분할
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = splitter.split_documents(texts)

# 4. 임베딩 + Pinecone 연결
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
index = pc.Index(PINECONE_INDEX_NAME)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# 5. Pinecone에 업로드 (배치 처리)
BATCH_SIZE = 50  # 배치 사이즈 줄이기(예: 20~50)
for i in range(0, len(split_docs), BATCH_SIZE):
    batch = split_docs[i:i + BATCH_SIZE]
    success = False
    while not success:
        try:
            vector_store.add_documents(batch)
            success = True
        except RateLimitError as e:
            print("RateLimitError 발생! 10초 대기 후 재시도")
            time.sleep(10)
    print(f"Uploaded {i + len(batch)} / {len(split_docs)} documents")
    # 추가로 매 배치마다 2~3초 휴식(더 안정)
    time.sleep(2)


Uploaded 50 / 67346 documents
Uploaded 100 / 67346 documents
Uploaded 150 / 67346 documents
Uploaded 200 / 67346 documents
Uploaded 250 / 67346 documents
Uploaded 300 / 67346 documents
Uploaded 350 / 67346 documents
Uploaded 400 / 67346 documents
Uploaded 450 / 67346 documents
Uploaded 500 / 67346 documents
Uploaded 550 / 67346 documents
Uploaded 600 / 67346 documents
Uploaded 650 / 67346 documents
Uploaded 700 / 67346 documents
Uploaded 750 / 67346 documents
Uploaded 800 / 67346 documents
Uploaded 850 / 67346 documents
Uploaded 900 / 67346 documents
Uploaded 950 / 67346 documents
Uploaded 1000 / 67346 documents
Uploaded 1050 / 67346 documents
Uploaded 1100 / 67346 documents
Uploaded 1150 / 67346 documents
Uploaded 1200 / 67346 documents
Uploaded 1250 / 67346 documents
Uploaded 1300 / 67346 documents
Uploaded 1350 / 67346 documents
Uploaded 1400 / 67346 documents
Uploaded 1450 / 67346 documents
Uploaded 1500 / 67346 documents
Uploaded 1550 / 67346 documents
Uploaded 1600 / 67346 docume

PineconeApiException: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 15 Jul 2025 06:35:53 GMT', 'Content-Type': 'application/json', 'Content-Length': '166', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '236', 'x-pinecone-request-id': '7930506515639808871', 'x-envoy-upstream-service-time': '3', 'server': 'envoy'})
HTTP response body: {"code":8,"message":"Request failed. You've reached your write unit limit for the current month (2000000). To continue writing data, upgrade your plan.","details":[]}


In [35]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

# 임베딩 & 파인콘 세팅
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

vector_store = PineconeVectorStore(index=index, embedding=embeddings)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})  # Top-5 리트리브

# 사용자 쿼리 예시
query = "삼성전자 2023년 사업보고서에서 반도체 매출 성장률은?"
retrieved_docs = retriever.invoke(query)
context = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [36]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import RegexParser

prompt_beginner = PromptTemplate.from_template("""
[초급 안내]
- 아래 문서에서만 정보를 찾아서, 학생이 쉽게 이해할 수 있도록 답하세요.
- 찾지 못하면 "문서에 없음"이라고 답하세요.

[문서 내용]
{context}

[질문]
{question}

[예시]
Q: '삼성전자 매출'이 뭐예요?
A: 삼성전자의 매출은 문서에 나온 총 판매 금액입니다.

Q: {question}
A:
""")


In [37]:
prompt_intermediate = PromptTemplate.from_template("""
[중급 안내]
- 아래 문서에서만 정보를 뽑아 정확하고 간결하게 답하세요.
- 문서에 없으면 "문서에 근거 없음"이라고 답하세요.

[문서]
{context}

[질문]
{question}

[예시]
Q: '반도체 성장률'이?
A: 삼성전자 반도체 부문의 성장률은 XX%입니다.

Q: {question}
A:
""")


In [57]:
prompt_advanced = PromptTemplate.from_template("""
[고급 안내]
- 아래 문서 근거를 바탕으로, 수치·인용까지 명확하게 작성하세요.
- 답변에 반드시 문서 내 수치·근거를 포함하세요.
- 근거가 없으면 "정보 없음"이라고만 답하세요.

[문서]
{context}

[질문]
{question}

[예시]
Q: '삼성전자 반도체 매출과 성장률은?'
A: 2023년 삼성전자 반도체 부문 매출은 80조원(문서 12p 기준), 성장률은 전년 대비 6.5%입니다.

Q: {question}
A:
""")


In [59]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import RegexParser

# LLM, 파서 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
parser = RegexParser(regex=r'(?s)(.*)', output_keys=['answer'])

# 원하는 난이도에 따라 프롬프트 선택
selected_prompt = prompt_advanced    # 또는 prompt_beginner, prompt_intermediate

# 체인 객체 생성
selected_chain = selected_prompt | llm | parser


In [60]:
# ================ #
# 질문 → 답변 함수 #
# ================ #

def ask(query, retriever=None, chain=None, k=4, print_docs=False):
    """
    - query: 사용자가 질문
    - retriever: bm25_retriever 또는 dense_retriever 등 (None이면 bm25 사용)
    - chain: langchain 체인 객체 (None이면 selected_chain 사용)
    - k: Top-K 문서 개수
    - print_docs: True면 문서도 같이 보여줌(디버깅용)
    """
    # 1. Retriever로 top-k 문서 가져오기
    _retriever = retriever or bm25_retriever
    top_docs = _retriever.invoke(query)
    if print_docs:
        print("==== Retrieved Docs ====")
        for i, doc in enumerate(top_docs):
            print(f"[{i+1}] {doc.page_content[:300]}")
        print("="*30)
    # 2. context 구성 (긴 문서면 자르기 등 필요하면 추가)
    context = "\n\n".join(doc.page_content for doc in top_docs)
    # 3. 프롬프트 체인 실행 (selected_chain 또는 chain)
    _chain = chain or selected_chain
    response = _chain.invoke({
        "context": context,
        "question": query
    })
    # 4. 답변만 출력
    return response.get("answer", response)

# ========== #
# 사용 예시  #
# ========== #
print(ask("네이버의 주요 사업 부문은 어떻게 구성되어 있어?"))
print(ask("2024년 삼성전자 사업보고서에서 영업이익은 얼마인가?"))
print(ask("2023년 SK하이닉스 사업보고서에서 임원 명단을 보여줘."))

# print_docs=True로 디버깅도 가능
# print(ask("네이버의 주요 사업 부문은?", print_docs=True))

정보 없음
2024년 삼성전자의 영업이익은 14조 396억원입니다. (문서 기준)
정보 없음
